In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
import PyPDF2
from PyPDF2 import PdfFileReader
import re
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
import sys
from sklearn import linear_model
warnings.filterwarnings("ignore")
sys.path.append(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\asc-library-temp')

In [2]:
from library import Dataset
from finbert_senti import get_finbert_sentiments

In [3]:
driver = webdriver.Chrome(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\chromedriver.exe')

In [43]:
while True:
    try:
        button = driver.find_element_by_link_text('View more')
        button.click()
        time.sleep(0.5)
    except:
        break

In [78]:
elems = driver.find_elements_by_xpath("//a[@href]")
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))
useful_links = [link for link in links if link.find('researchId') != -1]
df = pd.DataFrame(useful_links)
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [79]:
for i in range(0,len(df)):
    driver.get(df.iloc[i,0])
    time.sleep(3)
    button = driver.find_element_by_link_text('DOWNLOAD')
    button.click()
    
    time.sleep(0.5)

In [27]:
path = r"C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\Macquarie trades\*.pdf"
files=glob.glob(path)
reports_path = []
for file_name in files:
    reports_path.append(file_name)
df = pd.DataFrame(reports_path, columns =['Path'])

In [28]:
df['Text'] = ''
for i in range(0,len(df)):
    path = df.loc[i,'Path']
    pdf=PdfFileReader(path)
    text=pdf.getPage(0).extractText()
    txt = text.replace('\n','')
    df.loc[i,'Text'] = txt
df[df['Text']=='']

,Path,Text


In [29]:
for i in range(0,len(df)):
    try:
        path = df.loc[i,'Path']
        pdf=PdfFileReader(path)
        text=pdf.getPage(-1).extractText()
        txt = text.replace('\n','')
        df.loc[i,'publish_date_and_time'] = txt
    except:
        print(i)
    time = df.loc[i,'publish_date_and_time']
    time= time.replace('This publication was disseminated on ','')
    time = time.replace('at ','')
    time = time.replace('.','')
    time= time.replace(' UTC','')
    df.loc[i,'publish_date_and_time'] = time
df[df['publish_date_and_time']=='']

,Path,Text,publish_date_and_time


In [30]:
#Analyst
df['Primary Analyst'] = ''

for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    #primary analyst
    txt_lst = txt.split()
    emails = [i for i in txt_lst if '@macquarie.com' in i]
    emails = [i.replace('@macquarie.com', '') for i in emails]
    emails = [i.split('.') for i in emails]
    if len(emails) > 0:
        analyst_pri = [i.capitalize() for i in emails[0]]
        analyst_pri = ' '.join(analyst_pri)
        df.loc[x,'Primary Analyst'] = analyst_pri
df['Primary Analyst'].unique()

array(['Terence Chang', 'Cherry Ma', 'Linda Huang'], dtype=object)

In [31]:
df['ticker'] = ''
for i in range(0,len(df)):
    txt = df.loc[i,'Text']
    namere=re.compile(r'@macquarie.com\s+([A-ZÉa-z0-9&-( ]+)\)\s')
    name=namere.findall(txt)
    ticker = ''
    company_name = ''
    if len(name) >0:
        if len(name) >1:
            name = name[-1]
        else:
            name = name[0]
        name = name.split(' ')
        tick = ' '.join([name[-2],name[-1]])
        ticker = tick.replace('(','')
        ticker = ticker.replace(')','')
        company_name = name[:len(name)-2]
        company_name = ' '.join(company_name)
    df.loc[i,'Company Name'] = company_name
    df.loc[i,'ticker'] = ticker
    if df.loc[i,'ticker'] == '':
        tickre = re.compile(r'\d{3,4}\s[HK|JP|TT]')
        ticker = tickre.findall(txt)
        if len(ticker)>0:
            ticker = ticker[0]
        else:
            ticker = ''
    df.loc[i,'ticker'] = ticker
df[df['ticker']=='']
df['ticker'].unique()

array(['2020 HK', '285 HK', '6808 HK'], dtype=object)

In [32]:
#Current Rating
df['Current Rating'] = ''
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    r = re.compile('[JP|HK|TT]\s+([A-Za-z]+)\sPrice\s')
    rating_curr = r.findall(txt)
    if len(rating_curr)>0:
        rating_curr = rating_curr[0]
    else:
        rating_curr = ''
    df.loc[x,'Current Rating'] = rating_curr
df[df['Current Rating']=='']

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating


In [33]:
#Headline
df['headline'] = ''
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    #Headline
    hl_re = re.compile(r'\s([~A-za-z0-9 +&\.,;:%/¥$?-]+)\s(?:Event\s+|Key\spoints\s+)')
    headline = hl_re.findall(txt)
    if len(headline) > 0:
        headline = headline[0]
    else:
        hl_re = re.compile(r'\s([~A-za-z0-9 +&\.,;:%/¥$?-]+)\sConclusion\s+')
        headline = hl_re.findall(txt)
        if len(headline) > 0:
            headline = headline[0]
        else:
            hl_re = re.compile(r'[JP|HK|TT]\)([~A-za-z0-9-; ]+)')
            headline = hl_re.findall(txt)
            if len(headline) > 0:
                headline = headline[0]
            else:
                headline = ''
    df.loc[x,'headline'] = headline
df[df['headline']=='']

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating,headline


In [34]:
#Summary
df['summary'] = ''
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    smr = re.compile(r'@macquarie.com\s+([A-Za-z0-9-()/;,.:$¥%+>&?! ]+)')
    smr = smr.findall(txt)
    if len(smr) > 0:
        if len(smr) == 1:
            smr = smr[0]
        else:
            smr = smr[-1]
    else:
        smr = ''
    df.loc[x,'summary'] = smr
df[df['summary']=='']   

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating,headline,summary


In [35]:
#GICS
df['gics1'] = ''
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    gics = re.compile(r'GICS\ssector\s+([A-Za-z-()/,& ]+)\s+Mar')
    gics1 = gics.findall(txt)
    if len(gics1) > 0:
        gics1 = gics1[0]
    else:
        gics1 = ''
    df.loc[x,'gics1'] = gics1
df[df['gics1']=='']   

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating,headline,summary,gics1


In [36]:
#Report type
df['Report Type'] = 'Ad hoc'
for x in range(0,len(df)):
    txt = df.loc[x,'Text']
    er = re.compile(r'(Strong|strong|Good|good)\s+[~A-za-z0-9 +&\.,;:%/¥$?-]+results')
    er = er.findall(txt)
    if len(er)>0:
        df.loc[x,'Report Type'] = 'Earning Review'
    er = re.compile(r'(Beat|beat)\s+[~A-za-z0-9 +&\.,;:%/¥$?-]+consensus')
    er = er.findall(txt)
    if len(er)>0:
        df.loc[x,'Report Type'] = 'Earning Review'
    
    if df.loc[x,'Report Type'] != 'Earning Review':
        txt = df.loc[x,'headline']
        substrings = ['3Q','1Q','2Q','4Q','1H','2H','FY','2018','2019','2020','2021','earnings','Earnings']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] = 'Earning Related'
        else:
            txt = df.loc[x,'Text']
            substrings = ['Initiate','initiate coverage','Initiating','initiating']
            if any(substring in txt for substring in substrings)== True:
                df.loc[x,'Report Type'] = 'Initiation'
            else: 
                substrings = ['Upgrade to','upgrade to','Upgrade from','upgrade from','Upgrading','upgrading'] 
                if any(substring in txt for substring in substrings)== True:
                    df.loc[x,'Report Type'] = 'Rating Upgrade'
                else:
                    substrings = ['Downgrade to','downgrade to','Downgrade from','downgrade from']
                    if any(substring in txt for substring in substrings)== True:
                        df.loc[x,'Report Type'] = 'Rating Downgrade'   
                    else:
                        substrings = ['Earnings and target price revision']
                        if any(substring in txt for substring in substrings)== True:
                            df.loc[x,'Report Type'] = 'TP Related and Estimate Change'

    if df.loc[x,'Report Type'] == 'Ad hoc':
        txt = df.loc[x,'Text']
        substrings = ['Initiate','initiate','Initiating','initiating']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] = 'Initiation'
        else: 
            substrings = ['Upgrade to','upgrade to','Upgrade from','upgrade from'] 
            if any(substring in txt for substring in substrings)== True:
                df.loc[x,'Report Type'] = 'Rating Upgrade'
            else:
                substrings = ['Downgrade to','downgrade to','Downgrade from','downgrade from']
                if any(substring in txt for substring in substrings)== True:
                    df.loc[x,'Report Type'] = 'Rating Downgrade'   
                else:
                    substrings = ['Earnings and target price revision']
                    if any(substring in txt for substring in substrings)== True:
                        df.loc[x,'Report Type'] = 'TP Related and Estimate Change'
                        
    
    if df.loc[x,'Report Type'] == 'Earning Related':
        txt = df.loc[x,'summary']
        substrings = ['beat','miss','in line','above','below','In line','Above','Below','Beat','Miss','remains','remain','beats','misses','Beats','Misses','earnings','Earnings''results were better']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] = 'Earning Review'
            
        else:
            txt = df.loc[x,'Text']
            substrings = ['Initiate','initiate','Initiating','initiating']
            if any(substring in txt for substring in substrings)== True:
                df.loc[x,'Report Type'] = 'Initiation'
            else: 
                substrings = ['Upgrade to','upgrade to','Upgrade from','upgrade from'] 
                if any(substring in txt for substring in substrings)== True:
                    df.loc[x,'Report Type'] = 'Rating Upgrade'
                else:
                    substrings = ['Downgrade to','downgrade to','Downgrade from','downgrade from']
                    if any(substring in txt for substring in substrings)== True:
                        df.loc[x,'Report Type'] = 'Rating Downgrade'   
                    else:
                        substrings = ['overshot the consensus']
                        if any(substring in txt for substring in substrings)== True:
                            df.loc[x,'Report Type'] = 'Earning Review'  
                        else:
                            df.loc[x,'Report Type'] = 'Ad hoc'
                            
    if df.loc[x,'Report Type'] == 'TP Related and Estimate Change':
        txt = df.loc[x,'Text']
        tp = re.compile(r'(Earnings and target price revision)\s+[~A-za-z0-9 +&\.,;:%/¥$?-]+')
        tp = tp.findall(txt)[0]
        substrings1 = ['TP','Target Price','target price','Target price']
        substrings2 = ['unchanged target price']
        if any(substring in tp for substring in substrings1)== True and any(substring in tp for substring in substrings2)== False:
            df.loc[x,'Report Type'] = 'TP change'
        else:
            df.loc[x,'Report Type'] = 'Estimate change'
        substrings = ['No change']
        if any(substring in txt for substring in substrings)== True:
            df.loc[x,'Report Type'] ='Ad hoc'
            
            
                            
df[df['Report Type']=='']

,Path,Text,publish_date_and_time,Primary Analyst,ticker,Company Name,Current Rating,headline,summary,gics1,Report Type


In [37]:
data = Dataset(df)
data.df['headline_senti'] = get_finbert_sentiments(data.df['headline'])
data.df['summary_senti'] = get_finbert_sentiments(data.df['summary'])
data.clean(tz_name='Europe/London')
df = data.df

Tickers [['2020 HK Equity', '2022-04-18'], ['285 HK Equity', '2022-04-18'], ['6808 HK Equity', '2022-04-19']] are in the trading days json file, but cannot return an entry date.


In [2]:
df1 = pd.read_csv(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\trades\scoring.csv')

NameError: name 'pd' is not defined

In [39]:
df['side'] = ''
for x in range(len(df)):
    if df.loc[x,'summary_senti'] == 'positive' and df.loc[x,'headline_senti'] == 'positive':
        df.loc[x,'side'] = 'long'
    elif df.loc[x,'summary_senti'] == 'negative' or df.loc[x,'headline_senti'] == 'negative':
        df.loc[x,'side'] = 'short'
df = df.dropna(subset=['side'])
df = df.reset_index(drop=True)

In [40]:
for i in range(len(df)):
    if df.loc[i,'side'] == 'long':
        if df.loc[i,'Current Rating'] == 'Outperform':
            df.loc[i,'w5'] = 1
        else:
            df.loc[i,'w5'] = 0
    else:
        if df.loc[i,'Current Rating'] == 'Outperform':
            df.loc[i,'w5'] = 0
        else:
            df.loc[i,'w5'] = 1

In [41]:
df1 = pd.concat([df1,df])

In [1]:
df1

NameError: name 'df1' is not defined

In [42]:
dft = df[df['exch_location']=='Taiwan'].reset_index(drop=True)
dfh = df1[df1['exch_location']=='Hong Kong'].reset_index(drop=True)
#   dft = df[df['Country']=='TT'].reset_index(drop=True)
dfj = df1[df1['exch_location']=='Japan'].reset_index(drop=True)
#   dfk = df[df['Country']=='KS'].reset_index(drop=True)
allset = [dfh,dfj]
predset= [dfh[dfh['report_id'].isnull()==False],dfj[dfj['report_id'].isnull()==False]]
valset= [dfh[dfh['report_id'].isnull()==True],dfj[dfj['report_id'].isnull()==True]]
vali_sets = [] 
for i in range(len(predset)):

    lst = list(allset[i]['side'].unique())
    w1 = []
    for x in lst:
        pf = predset[i][predset[i]['side']== x]
        num = pf['d0_r'].mean()
        w1.append(num)
    zip_iterator = zip(lst, w1)
    a_dictionary = dict(zip_iterator)
    for x in range(len(allset[i])):
        allset[i].loc[x,'w1'] = a_dictionary[allset[i].loc[x,'side']]


    lst = list(allset[i]['Primary Analyst'].unique())
    w2 = []
    for x in lst:
        pf = predset[i][predset[i]['Primary Analyst']== x]
        num = pf['d0_r'].mean()
        w2.append(num)
    zip_iterator = zip(lst, w2)
    a_dictionary = dict(zip_iterator)
    for x in range(len(allset[i])):
        allset[i].loc[x,'w2'] = a_dictionary[allset[i].loc[x,'Primary Analyst']]

    lst = list(allset[i]['Report Type'].unique())
    w3 = []
    for x in lst:
        pf = predset[i][predset[i]['Report Type']== x]
        num = pf['d0_r'].mean()
        w3.append(num)
    zip_iterator = zip(lst, w3)
    a_dictionary = dict(zip_iterator)
    for x in range(len(allset[i])):
        allset[i].loc[x,'w3'] = a_dictionary[allset[i].loc[x,'Report Type']]

    lst = list(allset[i]['gics1'].unique())
    w4 = []
    for x in lst:
        pf = predset[i][predset[i]['gics1']== x]
        num = pf['d0_r'].mean()
        w4.append(num)
    zip_iterator = zip(lst, w4)
    a_dictionary = dict(zip_iterator)
    for x in range(len(allset[i])):
        allset[i].loc[x,'w4'] = a_dictionary[allset[i].loc[x,'gics1']]


    lst = list(allset[i]['release_period'].unique())
    w6 = []
    for x in lst:
        pf = predset[i][predset[i]['release_period']== x]
        num = pf['d0_r'].mean()
        w6.append(num)
    zip_iterator = zip(lst, w6)
    a_dictionary = dict(zip_iterator)
    for x in range(len(allset[i])):
        allset[i].loc[x,'w6'] = a_dictionary[allset[i].loc[x,'release_period']]


    allset[i] = allset[i].fillna(value = { "w1": allset[i].loc[:,'w1'].mean(), "w2": allset[i].loc[:,'w2'].mean(), "w3": allset[i].loc[:,'w3'].mean(),
                                "w4": allset[i].loc[:,'w4'].mean(), "w5": allset[i].loc[:,'w5'].mean(),"w6": allset[i].loc[:,'w6'].mean()})
    
    dd = allset[i]
    predset= [dd[dd['report_id'].isnull()==False],dd[dd['report_id'].isnull()==False]]
    valset= [dd[dd['report_id'].isnull()==True],dd[dd['report_id'].isnull()==True]]


    Y = predset[i].loc[:,'d0_r_af']
    X = predset[i].loc[:,['w1','w2','w3','w4','w5','w6']]
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)
    try:
        predicted_r = regr.predict(valset[i].loc[:,['w1','w2','w3','w4','w5','w6']])
        sort_by = list(predicted_r)
        valset[i] = valset[i].reset_index(drop=True)
        for x in range(len(valset[i])):
            valset[i].loc[x,'sort_by'] = predicted_r[x]
        vali_sets.append(valset[i])
    except:
        print('No trade')
vali_sets.append(dft)
val = pd.concat(vali_sets).reset_index(drop=True)
data= Dataset(val)
#trades = data._select_trades_with_portfolio_constraint(data.df, sort_by='sort_by',max_trade=2,max_skew=0).reset_index(drop=True)
#trades = trades[['ticker_updated','sort_by','side','headline_senti','summary_senti','Company Name','Primary Analyst','Report Type','gics1','release_period','headline','summary','publish_date_and_time']]
val = val[['ticker','date_local','release_period','side','gics1','Report Type','headline_senti','summary_senti',
           'headline','summary','Current Rating','Target Price','sort_by','Primary Analyst','exch_location']].reset_index(drop=True)

No trade


In [43]:
val

,ticker,date_local,release_period,side,gics1,Report Type,headline_senti,summary_senti,headline,summary,Current Rating,Target Price,sort_by,Primary Analyst,exch_location
0,2020 HK,2022-04-18,Within,short,Consumer Durables & Apparel,Earning Review,negative,negative,COVID lockdown impact dragging into 2Q22,Anta Sports (2020 HK) COVID lockdown impact dr...,Outperform,NaN,0.068952,Terence Chang,Hong Kong
1,285 HK,2022-04-18,After,short,Technology Hardware & Equipment,Earning Review,neutral,negative,HK,BYD Electronic (285 HK) Consumption slowdown a...,Outperform,NaN,0.028715,Cherry Ma,Hong Kong
2,6808 HK,2022-04-19,Before,short,Food & Staples Retailing,Earning Review,positive,negative,FY22 profit warning,Sun Art Retail Group (6808 HK) FY22 profit war...,Outperform,NaN,0.387988,Linda Huang,Hong Kong


In [44]:
val = val.rename(columns={"ticker": "Ticker", 
                          "date_local": "Date of publish",
                         "release_period": "Timing",
                         "side": "Side",
                         "sector": "gics1", 
                         "Report Type": "Report Type (Broker)",
                         "headline_senti": "Headline Sentiment",
                         "summary_senti": "Summary Sentiment",
                          "Primary Analyst": "Analyst Primary",
                         "headline": "Headline Content",
                         "summary": "Summary Content",
                         "Target Price": "Current Target Price",
                         "sort_by": "Final Expectancy D0",
                         "long": "Top Analyst (long)",
                        "short": "Top Analyst (short)",
                        "analyst_pri": "Analyst Primary"
                         })

In [45]:
bench= pd.read_csv(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\Macquarie New\mac_full.csv')


In [46]:
for i in range(len(val)):
    needed = bench[(bench['headline_senti']==val.loc[i,'Headline Sentiment'])&(bench['summary_senti']==val.loc[i,'Summary Sentiment'])&(bench['Report Type']==val.loc[i,'Report Type (Broker)'])&(bench['exch_location']==val.loc[i,'exch_location'])]
    val.loc[i,'Blind Long expectancy'] = needed.dropna(subset=['l0'])['l0'].mean()
    val.loc[i,'Blind Short expectancy'] = needed.dropna(subset=['s0'])['s0'].mean()

In [47]:
anal = pd.read_csv(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\mac_anl.csv')
val = val.merge(anal, on="Analyst Primary", how='left')
val[['Top Analyst (long)', 'Top Analyst (short)']] = val[['Top Analyst (long)', 'Top Analyst (short)']].fillna(False)
val['Broker']='Macquarie'

In [48]:
from datetime import datetime 

In [49]:
val.to_csv(r'C:\Users\ZeyuWang\OneDrive - Alpha Sherpa Capital\Desktop\MAC'+datetime.today().strftime('%m%d')+'.csv',index = False)

In [197]:
datetime.today()

datetime.datetime(2022, 4, 12, 8, 25, 18, 190779)